In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
openD = pd.read_csv('data/open.csv', header = None)
closeD = pd.read_csv('data/close.csv', header = None)
silenceD = pd.read_csv('data/silence.csv', header = None)
print(openD)
print(closeD)
print(silenceD)

         0     1     2     3     4    5    6    7
0     25.0  49.0  48.0  75.0  18.0  1.0  0.0  4.0
1     28.0  50.0  46.0  86.0  69.0  4.0  0.0  7.0
2     24.0  44.0  46.0  82.0  42.0  2.0  0.0  4.0
3     23.0  41.0  56.0  84.0  38.0  3.0  0.0  4.0
4     19.0  34.0  55.0  50.0  27.0  2.0  0.0  3.0
...    ...   ...   ...   ...   ...  ...  ...  ...
2995   1.0   1.0   0.0   0.0   0.0  0.0  0.0  0.0
2996   0.0   0.0   0.0   0.0   0.0  0.0  0.0  0.0
2997   0.0   0.0   0.0   0.0   0.0  0.0  0.0  0.0
2998   1.0   0.0   0.0   0.0   0.0  0.0  0.0  0.0
2999   0.0   1.0   0.0   0.0   0.0  0.0  0.0  0.0

[3000 rows x 8 columns]
         0     1      2      3     4    5    6     7
0     21.0  78.0   49.0   24.0   2.0  0.0  0.0   1.0
1     22.0  75.0   98.0   37.0   4.0  1.0  0.0   3.0
2     15.0  58.0  128.0   87.0   6.0  1.0  0.0   6.0
3     13.0  45.0  128.0  128.0  13.0  3.0  0.0  11.0
4     14.0  33.0  128.0   95.0  25.0  3.0  0.0   8.0
...    ...   ...    ...    ...   ...  ...  ...   ...
2995

In [3]:
SAMPLES_PER_VOICE = 30
FEATURE_NUM = 8

In [4]:
def processData(d, v):
    dataX = np.empty([0,SAMPLES_PER_VOICE*FEATURE_NUM])
    dataY = np.empty([0])

    data  = d.values
    dataNum = data.shape[0] // SAMPLES_PER_VOICE

    for i in tqdm(range(dataNum)):
        tmp = []
        for j in range(SAMPLES_PER_VOICE):
            tmp += [data[i * SAMPLES_PER_VOICE + j][0] / 128.0]
            tmp += [data[i * SAMPLES_PER_VOICE + j][1] / 128.0]
            tmp += [data[i * SAMPLES_PER_VOICE + j][2] / 128.0]
            tmp += [data[i * SAMPLES_PER_VOICE + j][3] / 128.0]
            tmp += [data[i * SAMPLES_PER_VOICE + j][4] / 128.0]
            tmp += [data[i * SAMPLES_PER_VOICE + j][5] / 128.0]
            tmp += [data[i * SAMPLES_PER_VOICE + j][6] / 128.0]
            tmp += [data[i * SAMPLES_PER_VOICE + j][7] / 128.0]

        tmp = np.array(tmp)
        tmp = np.expand_dims(tmp, axis = 0)

        dataX = np.concatenate((dataX, tmp), axis = 0)
        dataY = np.append(dataY, v)

    return dataX, dataY

In [5]:
silenceX, silenceY = processData(silenceD, 0)
openX, openY = processData(openD, 1)
closeX, closeY = processData(closeD, 2)

100%|██████████| 100/100 [00:00<00:00, 4238.21it/s]


In [6]:
dataX = np.concatenate((silenceX, openX), axis = 0)
dataY = np.append(silenceY, openY)
dataX = np.concatenate((dataX, closeX), axis = 0)
dataY = np.append(dataY, closeY)
print(dataX)
print(dataY)

[[0.015625  0.        0.        ... 0.        0.        0.       ]
 [0.0078125 0.        0.        ... 0.        0.        0.       ]
 [0.0078125 0.        0.        ... 0.        0.        0.       ]
 ...
 [0.203125  0.5       0.4609375 ... 0.        0.        0.       ]
 [0.1015625 0.234375  0.140625  ... 0.        0.        0.       ]
 [0.1953125 0.671875  0.46875   ... 0.        0.        0.       ]]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1

In [7]:
permutationTrain = np.random.permutation(dataX.shape[0])
print(permutationTrain)

[163 100 182 166 154  97  10 133 251 214 161   9 248 169  44 230 146  96
 288 260 134 206 138  75  35 273 183  42 102 207 254 271 246 216 109  29
   2 264  78  60  83 131  70 150  79  86 261  71 107 235  51 201 152 237
 139 122  13 275 242 170  65  74 236   1 135 199  18 268 111 114 267 173
 189 123  48  90 128 210 215 177  73 258 118  46 155 279 175  52 259  24
 234 269  84  16 140  50 283  72 136 110  62 143 121 276  54 164 272 231
 149 198 186  26 151 200 142 263 297 228 192  53  76 294  22 226 190 213
  15 167 296 245 253 179 185 172  64 252  43 125 188 227 116 217 187 132
  69 148 147 223  94  19 299 162 103 262 137  55 126 244  63 247 239  82
 280 184  87  85  32  57  61 129 287  68 174  67 193 204 295 196 222  95
 130 274   4  66 191 168 298 229  27 144  36 221 113  77   3 141 112 282
 209   5   6 176 106  49 218 194 256 219 250 180  92 255 115 211   0 181
 108  88 171  37   8 165  30 153 241 202 157 224 277 290 159 293  28  98
 270  23 119  81 285 101  56 281 145 178  12  38 15

In [8]:
dataX = dataX[permutationTrain]
dataY = dataY[permutationTrain]
print(dataY)

[1. 1. 1. 1. 1. 0. 0. 1. 2. 2. 1. 0. 2. 1. 0. 2. 1. 0. 2. 2. 1. 2. 1. 0.
 0. 2. 1. 0. 1. 2. 2. 2. 2. 2. 1. 0. 0. 2. 0. 0. 0. 1. 0. 1. 0. 0. 2. 0.
 1. 2. 0. 2. 1. 2. 1. 1. 0. 2. 2. 1. 0. 0. 2. 0. 1. 1. 0. 2. 1. 1. 2. 1.
 1. 1. 0. 0. 1. 2. 2. 1. 0. 2. 1. 0. 1. 2. 1. 0. 2. 0. 2. 2. 0. 0. 1. 0.
 2. 0. 1. 1. 0. 1. 1. 2. 0. 1. 2. 2. 1. 1. 1. 0. 1. 2. 1. 2. 2. 2. 1. 0.
 0. 2. 0. 2. 1. 2. 0. 1. 2. 2. 2. 1. 1. 1. 0. 2. 0. 1. 1. 2. 1. 2. 1. 1.
 0. 1. 1. 2. 0. 0. 2. 1. 1. 2. 1. 0. 1. 2. 0. 2. 2. 0. 2. 1. 0. 0. 0. 0.
 0. 1. 2. 0. 1. 0. 1. 2. 2. 1. 2. 0. 1. 2. 0. 0. 1. 1. 2. 2. 0. 1. 0. 2.
 1. 0. 0. 1. 1. 2. 2. 0. 0. 1. 1. 0. 2. 1. 2. 2. 2. 1. 0. 2. 1. 2. 0. 1.
 1. 0. 1. 0. 0. 1. 0. 1. 2. 2. 1. 2. 2. 2. 1. 2. 0. 0. 2. 0. 1. 0. 2. 1.
 0. 2. 1. 1. 0. 0. 1. 2. 0. 0. 0. 2. 2. 0. 1. 0. 0. 1. 0. 1. 2. 0. 0. 2.
 1. 0. 2. 0. 2. 2. 0. 0. 1. 1. 2. 0. 1. 2. 0. 0. 2. 2. 2. 0. 1. 0. 0. 2.
 2. 1. 2. 0. 0. 2. 2. 2. 0. 2. 1. 2.]


In [9]:
vfoldSize = int(dataX.shape[0]/100*20)

xTest = dataX[0:vfoldSize]
yTest = dataY[0:vfoldSize]

xTrain = dataX[vfoldSize:dataX.shape[0]]
yTrain = dataY[vfoldSize:dataY.shape[0]]

In [10]:
model = keras.Sequential()

In [11]:
model.add(keras.layers.Dense(32, input_shape =(FEATURE_NUM*SAMPLES_PER_VOICE,), activation='relu'))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

In [12]:
adam = keras.optimizers.Adam(0.00001)

In [13]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=adam,
              metrics=['sparse_categorical_accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                7712      
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 51        
Total params: 8,291
Trainable params: 8,291
Non-trainable params: 0
_________________________________________________________________


In [15]:
history = model.fit(xTrain, yTrain, batch_size=1, validation_data=(xTest, yTest), epochs=1000, verbose=1)

Train on 240 samples, validate on 60 samples
Epoch 1/1000
240/240 [==============================] - 2s 8ms/sample - loss: 1.0628 - sparse_categorical_accuracy: 0.3750 - val_loss: 1.0584 - val_sparse_categorical_accuracy: 0.4333
Epoch 2/1000
240/240 [==============================] - 1s 4ms/sample - loss: 1.0554 - sparse_categorical_accuracy: 0.4583 - val_loss: 1.0514 - val_sparse_categorical_accuracy: 0.4833
Epoch 3/1000
240/240 [==============================] - 1s 4ms/sample - loss: 1.0485 - sparse_categorical_accuracy: 0.5125 - val_loss: 1.0446 - val_sparse_categorical_accuracy: 0.5333
Epoch 4/1000
240/240 [==============================] - 1s 4ms/sample - loss: 1.0415 - sparse_categorical_accuracy: 0.5667 - val_loss: 1.0376 - val_sparse_categorical_accuracy: 0.5667
Epoch 5/1000
240/240 [==============================] - 1s 4ms/sample - loss: 1.0344 - sparse_categorical_accuracy: 0.5917 - val_loss: 1.0303 - val_sparse_categorical_accuracy: 0.6000
Epoch 6/1000
240/240 [=============

In [16]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

open("model", "wb").write(tflite_model)

34744

In [17]:
!xxd -i model >> model.h